# Analiza podatkov iz Phyphox-a

In [81]:
%pip install -q bqplot

In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import bqplot.pyplot as plt
#%matplotlib inline
from scipy import signal
import sys,os
sys.path.append("../SPARC/scripts") # SPARC measure of movement smoothness
#from smoothness import log_dimensionless_jerk2 as ldlj

In [88]:
def Raw_Filtered_Plot(t,y,fy,measurement='Measurement',xlabel='x',ylabel='y',legend='True'):
    '''Plot Raw and Filtered data over time'''
    fig = plt.figure()
    plt.plot(t,y,label=f'Raw')# LDJ = {ldj:.0f}')
    plt.plot(t,fy,label=f'Filter')# LDJ = {ldjf:.0f}')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(measurement)
    if legend: plt.legend(loc='lower right')
    fig

In [89]:
def AnalyzeSignal(d,ydataNo,order=8,CNr=0.1,Start=100,Interval=0,plot=True,legend='True'):
    '''Get Raw and Filtered data over time and their Log_Dimensionless_Jerk'''
    if Interval == 0:
        Interval = len(d)-Start
    t, y = [d.iloc[:,col][Start:Start+Interval] for col in (0,ydataNo)]
    dtlist = t.diff()[1:]
    fslist = 1/dtlist # Sampling frequency
    fs, sfs = np.average(fslist),np.std(fslist)
    sampling = f', Sampling: $f_s$ = {fs:.0f} Hz, $\sigma (f_s) =$ {sfs:.0f} Hz'
    b, a = signal.butter(order, CNr * (0.5*fs), fs = fs) # Lowpass Butterforth filter
    fy = signal.filtfilt(b, a, y) # Apply filter forward and backward to a signal
   # ldj, ldjf = (ldlj(yy,fs,data_type='speed') for yy in (y,fy)) # Log_Dimensionless_Jerk

    Raw_Filtered_Plot(t,y,fy,
                      measurement,
                      xlabel=d.columns[0]+sampling,
                      ylabel=d.columns[ydataNo],
                      legend=legend)
    
    return t, y, fy

In [90]:
dirname = 'Acceleration with g 2023-10-21_14-00-55'
filename = 'Raw Data.csv'
#measurements = [f'{dirname}/{file}' for file in os.listdir(dirname)]
#measurements = [f'{dirname}/{filename}']
measurements = [f'{filename}']
print(measurements)

['Raw Data.csv']


In [91]:
pd.read_csv(measurements[0])

,Time (s),Acceleration x (m/s^2),Acceleration y (m/s^2),Acceleration z (m/s^2),Absolute acceleration (m/s^2)
0,0.019078,0.478565,9.635897,-1.071985,9.707146
1,0.021623,0.449851,9.643075,-1.093520,9.715300
2,0.024156,0.404387,9.645468,-1.119841,9.718674
3,0.026688,0.356531,9.655040,-1.136591,9.728245
4,0.029221,0.315853,9.669396,-1.160519,9.743911
...,...,...,...,...,...
17131,43.410774,2.658426,1.507478,9.466006,9.947110
17132,43.413307,2.651248,1.509871,9.449257,9.929616
17133,43.415840,2.658426,1.505085,9.444471,9.926256
17134,43.418373,2.677568,1.509871,9.451650,9.938952


In [93]:
ydataNo = 4 # Column Numbers to Analyze
order = 8 # Order of Butterworth filter
CNr = 0.05 # (0,1) f_crit/f_Nyquist in Butterworth filter. Try: 0.01 and 0.1.

for measurement in measurements:
    AnalyzeSignal(pd.read_csv(measurement),ydataNo,order,CNr,
                  Start=5700, # Start of the interval
                  Interval=4500,  # Interval length. Set 0 for whole measurement.
                  plot=True,legend=True)

In [106]:
%pip install -q ipywidgets

In [107]:
from ipywidgets import IntSlider

In [108]:
slider = IntSlider()

In [114]:
slider

IntSlider(value=44)

In [113]:
fig = plt.figure(title="test")
x = np.arange(100)
y = x-slider.value #np.cumsum(np.random.randn(100))
plt.plot(x,y)
fig

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

In [115]:
from ipywidgets import interact

In [121]:
def Interactive_Raw_Filtered_Plot(t,y,measurement='Measurement',xlabel='x',ylabel='y',legend='True'):
    '''Interactive Plot of Raw and Filtered data over time'''
    #fig = plt.figure()
    plt.plot(t,y,label=f'Raw')# LDJ = {ldj:.0f}')
    #plt.plot(t,fy,label=f'Filter')# LDJ = {ldjf:.0f}')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(measurement)
    if legend: plt.legend(loc='lower right')
    plt.show()

In [126]:
ydataNo = 4 # Column Numbers to Analyze

for measurement in measurements:
    d = pd.read_csv(measurement)

t, y = [d.iloc[:,col][Start:Start+Interval] for col in (0,ydataNo)]
Interactive_Raw_Filtered_Plot(t,y,fy,
                      measurement,
                      xlabel=d.columns[0]+sampling,
                      ylabel=d.columns[ydataNo],
                      legend=legend)

<class 'NameError'>: name 'Start' is not defined

In [122]:
interact(Interactive_Raw_Filtered_Plot)

<class 'ValueError'>: cannot find widget or abbreviation for argument: 't'

In [119]:
def interactive_plot(amplitude, frequency):
    x = np.linspace(0, 2 * np.pi, 1000)
    y = amplitude * np.sin(frequency * x)
    plt.figure(figsize=(8, 4))
    plt.plot(x, y)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title('Interactive Sine Wave')
    plt.grid(True)
    plt.show() 

In [120]:
 interact(interactive_plot, amplitude=(1, 5, 0.1), frequency=(1, 10, 0.1))

interactive(children=(FloatSlider(value=3.0, description='amplitude', max=5.0, min=1.0), FloatSlider(value=5.0…

<function __main__.interactive_plot(amplitude, frequency)>